In [69]:
import numpy as np
import math
import matplotlib.pyplot as plt
time_step = 0.03
#currently tracks to point (2,1,1)
J = np.array([[1.0,0.0,0.0],[0.0,1.0,0.0],[0.0,0.0,1.0]]) #inertia tensor
L = np.array([[1.0,0.0,0.0],[0.0,1.0,0.0],[0.0,0.0,1.0]]) #lambda parameter
K = 2*np.array([[1.0,0.0,0.0],[0.0,1.0,0.0],[0.0,0.0,1.0]]) #gain parameter

position trajectory functions 

In [50]:
def x_pos(t,x):
    return np.array([x[1], 4.0 - 3*x[1] - 2*x[0]])

def y_pos(t,y):
    return np.array([y[1], 2.0 - 3*y[1] - 2*y[0]])

def z_pos(t,z):
    return np.array([z[1], 2.0 - 3*z[1] - 2*z[0]])

In [51]:
#rk4 integrator

def rk4(func, tk, yk, dt):
    
    f1 = func(tk, yk)
    f2 = func(tk + dt / 2, yk + (f1 * (dt / 2)))
    f3 = func(tk + dt / 2, yk + (f2 * (dt / 2)))
    f4 = func(tk + dt, yk + (f3 * dt))

    return yk + (dt / 6) * (f1 + (2 * f2) + (2 * f3) + f4)


In [53]:
def trajectory_gen(time_now, present_state):
    
    x_present = [present_state[0],present_state[1]]
    y_present = [present_state[2],present_state[3]]
    z_present = [present_state[4],present_state[5]]
    time = time_now
    x_presentplusone = rk4(x_pos, time, x_present, time_step)
    y_presentplusone = rk4(y_pos, time, y_present, time_step)
    z_presentplusone = rk4(z_pos, time, z_present, time_step) 
    
    present_state[0] = x_presentplusone[0];
    present_state[1] = x_presentplusone[1];
    present_state[2] = y_presentplusone[0];
    present_state[3] = y_presentplusone[1];
    present_state[4] = z_presentplusone[0];
    present_state[5] = z_presentplusone[1];
    
    return present_state

attitude desired

In [54]:
def attitude_desired(present_state_derivative,q_d):
    
    X_bar = present_state_derivative[1]
    Y_bar = present_state_derivative[3]
    Z_bar = present_state_derivative[5] + 9.81
    
    Fd = np.sqrt((X_bar)**2 + (Y_bar)**2 + (Z_bar)**2)
    
    psi_d = 0
    phi_d = np.arcsin(-Y_bar/Fd)
    theta_d = np.arctan(X_bar/Z_bar)
    
    q_d[0] = theta_d
    q_d[1] = phi_d
    q_d[2] = psi_d
    return q_d

control block

In [55]:
def control(wr, wr_dot, w, u):
    
    
    cross = np.matmul(J,w)
    cross_skew = np.array([[0, -cross[2] , cross[1]],[cross[2], 0 , -cross[1]],[-cross[2], cross[1],0]])
    
    control_ip = np.matmul(J,wr_dot) - np.matmul(cross_skew,wr) - np.matmul(K,np.subtract(w , wr))
    
    u[0] = control_ip[0]
    u[1] = control_ip[1]
    u[2] = control_ip[2]
    return u

omega 

In [56]:
def omega(t,w):
    
    omega = np.array([w[0],w[1],w[2]])    
    control_ip = np.array([u[0],u[1],u[2]])
    #print('omega')
    #print(omega)
    #print('u')
    #print(control_ip)
    cross = np.matmul(J,omega)
    cross_skew = np.array([[0, -cross[2] , cross[1]],[cross[2], 0 , -cross[1]],[-cross[2], cross[1],0]])
    dw = np.matmul(cross_skew , omega) + control_ip
    dw = np.array([dw[0],dw[1],dw[2]])
    return dw

attitude q

In [57]:
def attitude(tk,q):
    
    theta = q[0]
    phi = q[1]
    psi = q[2]
    att = np.array([theta,phi,psi])
    omega = np.array([w[0],w[1],w[2]])
    Z_q = np.array([[1, np.sin(phi)*np.tan(theta), np.cos(phi)*np.tan(theta)],[0, np.cos(phi), -1*np.sin(phi)],[0, np.sin(phi)*(1/np.cos(theta)),np.cos(phi)*(1/np.cos(theta))]])
    
    dq = np.matmul(Z_q,omega)
    dq = np.array([dq[0],dq[1],dq[2]])
    return dq

reference omega

In [58]:
def ref_omega(q,q_d,q_ddot):
    
    theta = q[0]
    phi = q[1]
    psi = q[2]
    att = np.array([theta,phi,psi])
    omega_r = np.array([0,0,0])
    Z_q = np.array([[1, np.sin(phi)*np.tan(theta), np.cos(phi)*np.tan(theta)],[0, np.cos(phi), -1*np.sin(phi)],[0, np.sin(phi)*(1/np.cos(theta)),np.cos(phi)*(1/np.cos(theta))]])
    Z_q_inv = np.linalg.inv(Z_q)
   
    omega_r = np.matmul(Z_q_inv, q_ddot) + np.matmul(Z_q_inv, np.matmul(L,np.subtract(q_d,att)))
    

    return omega_r

In [59]:
def delay(time_delay):
    count = 0
    while (count<time_delay):
        count = count+1

In [71]:
initial_state = np.array([0.0,0.0,0.0,0.0,0.0,0.0])
present_state = initial_state
prev_state = np.array([0.0,0.0,0.0,0.0,0.0,0.0])
prev_qd = np.array([0.0,0.0,0.0]) 
q_d = np.array([0.0,0.0,0.0])#desired attitude
q_state = np.array([0.0,0.0,0.0])
w = np.array([0.0,0.0,0.0]) #omega
q = np.array([0.0,0.0,0.0])
wr = np.array([0.0,0.0,0.0])
prev_wr = np.array([0.0,0.0,0.0])
wr_dot = np.array([0.0,0.0,0.0])
u = np.array([0.0,0.0,0.0])

time_now = 0
while time_now < 30:
    time_now = time_now + time_step
    #print(present_state) 
    #print(present_state)
    
    prev_state[0] = present_state[0]
    prev_state[1] = present_state[1]
    prev_state[2] = present_state[2]
    prev_state[3] = present_state[3]
    prev_state[4] = present_state[4]
    prev_state[5] = present_state[5]
    
    present_state = trajectory_gen(time_now, present_state)
    present_state_derivative = np.subtract(present_state , prev_state)/time_step

    #print(present_state_derivative)
    prev_qd[0] = q_d[0]
    prev_qd[1] = q_d[1]
    prev_qd[2] = q_d[2]
    q_d = attitude_desired(present_state_derivative,q_d)
    
    q_ddot = np.subtract(q_d , prev_qd)/time_step   

    prev_wr = wr
    wr = ref_omega(q,q_d,q_ddot)
    
    wr_dot = np.subtract(wr,prev_wr)/time_step
    
    u = control(wr,wr_dot,w,u)
    
    w_state = np.array([w[0],w[1],w[2]])
    w_state = rk4(omega,time_now,w_state,time_step)
    w = [w_state[0],w_state[1],w_state[2]]
    
    q_state = rk4(attitude,time_now,q_state,time_step) 
    q = np.array([q_state[0],q_state[1],q_state[2]])
    
    